# 🧩 StructureLineage Quickstart (Colab)

This notebook demonstrates how to:
1. Clone the StructureLineage repo
2. Generate a synthetic pipeline
3. Build the Schema Dependency Graph (SDG)
4. Evaluate precision/recall against ground truth

In [ ]:
!git clone https://github.com/habiiibo03/StructureLineage.git
%cd StructureLineage
!pip install -r requirements.txt

In [ ]:
from src.tools.gen_synthetic import gen_project
gen_project('examples/synthetic_project_colab', n_tables=3, n_views=3)

In [ ]:
from src.sl_core.build_sdg import process_workspace
store = process_workspace('examples/synthetic_project_colab')
store.persist_json('examples/synthetic_project_colab/sdg.json')

In [ ]:
%%writefile src/tools/evaluate_sdg.py
import json
def load_ground_truth(path):
    with open(path) as f:
        return set(tuple(x) for x in json.load(f))
def load_sdg(path):
    with open(path) as f:
        j = json.load(f)
    return set((e['src'], e['tgt']) for e in j.get('edges', []))

In [ ]:
import sys
sys.path.append('/content/StructureLineage/src')
from tools.evaluate_sdg import load_ground_truth, load_sdg

gt = load_ground_truth('examples/synthetic_project_colab/ground_truth.json')
sdg = load_sdg('examples/synthetic_project_colab/sdg.json')

tp = len(gt & sdg)
fp = len(sdg - gt)
fn = len(gt - sdg)
prec = tp / (tp + fp) if tp + fp else 0
rec = tp / (tp + fn) if tp + fn else 0
f1 = 2 * prec * rec / (prec + rec) if prec + rec else 0

print(f'Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}')